# Summarize Text

## Overview

LLMs are a great tool for summarizing text content, given their proficiency in understanding and generating text.

In the context of [RAG (retrieval-augmented generation)](https://github.com/TCLee/rag-langchain), summarizing text can help distill the information in a large number of retrieved documents to provide context for a LLM.

In this notebook, we'll go over how to summarize text content with 3 different strategies. First, we split/chunk a long piece of text into "sub-documents". Then, we'll explore the following strategies:
- **Stuff**: Simply "stuff" all the documents into a single prompt. This is the simplest approach if all the documents can fit inside the model's context window.
- **Map-Reduce**: Summarize each document in a `map` step and then `reduce` the summaries into a final summary. The `map` step is typically _parallelized_ over the input documents.
- **Iterative Refinement**:
    1. Summarize the first document.
    2. Refine/update the result based on the next document.
    3. Repeat through the sequence of documents until finished.

## Run models locally

Before we begin, please make sure you followed the instructions in the [`README.md`](README.md) to setup [`Ollama`](https://ollama.com/) and [`Llama 3.2`](https://ollama.com/library/llama3.2).

Two important benefits of running LLMs locally on your own device are:
- **Privacy**: Your data is not sent to a third party, and it is not subject to the terms of service of a commercial service.
- **Cost**: There is no inference fee, which is important for token-intensive applications (e.g., summarization, agent simulations)

In this notebook, we'll be using `Llama 3.2` with size of `3B` parameters. We'll use `Ollama` to serve the LLM and run inference locally.

For more info, see the LangChain guide: [How to run models locally](https://python.langchain.com/docs/how_to/local_llms/).

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:3b", 
    temperature=0
)

llm.invoke("Hello. What can you do?")

AIMessage(content="I can be used in a variety of ways, from helping you plan a vacation to creating art. I'm here to assist you in finding the help or information you need. My strengths include answering questions, generating text and images and even just chatting with you.", additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-10-04T04:42:07.976979Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6827586375, 'load_duration': 2867225000, 'prompt_eval_count': 32, 'prompt_eval_duration': 211056000, 'eval_count': 53, 'eval_duration': 3742496000}, id='run-c87de788-f866-4089-8720-26a339b5306e-0', usage_metadata={'input_tokens': 32, 'output_tokens': 53, 'total_tokens': 85})